In [1]:
from fastai.vision.all import *

In [2]:
path = Path('/root/Documents/images')

In [3]:
path.ls()

(#144) [Path('/root/Documents/images/Amblyglyphidodon aureus'),Path('/root/Documents/images/Carassius auratus'),Path('/root/Documents/images/Abudefduf vaigiensis'),Path('/root/Documents/images/Cyprinella lutrensis'),Path('/root/Documents/images/Luxilus cornutus'),Path('/root/Documents/images/Chrysiptera taupou'),Path('/root/Documents/images/Phoxinus phoxinus'),Path('/root/Documents/images/Abudefduf sexfasciatus'),Path('/root/Documents/images/Notropis hudsonius'),Path('/root/Documents/images/Hesperoleucus symmetricus')...]

In [4]:
df_path = Path('/root/Documents/')

In [5]:
df = pd.read_csv(df_path/'csv/families_3.csv')

In [6]:
weights_df = pd.read_csv(df_path/'csv/families3-weights.csv')
weights_family_df = pd.read_csv(df_path/'csv/families3-weights-family.csv')

In [7]:
weights_df

,Specie,Count,Weight
0,Abramis brama,300,1.999722
1,Abudefduf abdominalis,177,3.389360
2,Abudefduf bengalensis,299,2.006410
3,Abudefduf saxatilis,1423,0.421586
4,Abudefduf septemfasciatus,128,4.686849
...,...,...,...
139,Stegastes partitus,174,3.447797
140,Stegastes planifrons,178,3.370318
141,Systomus rubripinnis,587,1.022005
142,Telestes muticellus,144,4.166088


# HD-CNN

In [8]:
n_classes = len(set(df['Specie']))

In [9]:
model = vgg16_bn
body = create_body(model, cut=-2)
body

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [10]:
head = create_head(512*2,n_classes)
head

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=1024, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=144, bias=False)
)

In [11]:
_body = body[0]
block1 = _body[:7]
block2 = _body[7:14]
block3 = _body[14:24]
block4 = _body[24:34]
block5 = _body[34:]

In [12]:
def custom_get_y(o):
    fine_label = o['Specie']
    coarse1_label = o['Family']
    return [coarse1_label, fine_label]

In [13]:
def count_classes_per_coarse(labels_df):
    result = {}
    for c in set(labels_df['coarse']):
        classes_with_c = labels_df[labels_df['coarse']==c]
        result[c] = len(classes_with_c)
    return result

In [14]:
labels_df = pd.DataFrame.from_dict({'fine': df['Specie'].values, 'coarse': df['Family'].values})
labels_df

,fine,coarse
0,Hypophthalmichthys molitrix,Cyprinidae
1,Dascyllus trimaculatus,Pomacentridae
2,Micropterus dolomieu,Centrarchidae
3,Cyprinus carpio,Cyprinidae
4,Abudefduf sordidus,Pomacentridae
...,...,...
86383,Pseudorasbora parva,Cyprinidae
86384,Parazacco spilurus,Cyprinidae
86385,Micropterus salmoides,Centrarchidae
86386,Lepomis macrochirus,Centrarchidae


In [15]:
classes_per_coarse = count_classes_per_coarse(labels_df)
classes_per_coarse

{'Cyprinidae': 28196, 'Pomacentridae': 19638, 'Centrarchidae': 38554}

In [16]:
class VGGCustomModel(Module):
    def __init__(self, model, classes_per_coarse):
        n_coarse_classes = len(classes_per_coarse.keys())
        self.shared_layers = create_body(model, cut=-2)[0][:24]
        self.coarse_component = create_body(model, cut=-2)[0][24:]
        self.coarse_head = create_head(512*2,n_coarse_classes)
        self.fine_components = nn.ModuleList([create_body(model, cut=-2)[0][24:] for _ in range(n_coarse_classes)])
        self.fine_heads = nn.ModuleList([create_head(512*2, 144) for num_classes in classes_per_coarse.values()])
        

    def forward(self, x):
        x = self.shared_layers(x)
        coarse_x = self.coarse_component(x)
        coarse1_label = self.coarse_head(coarse_x)
        coarse_idx = coarse1_label[0].argmax()
        x = self.fine_components[coarse_idx](x)
        fine_label = self.fine_heads[coarse_idx](x)
        return {
            'fine_label': fine_label,
            'coarse1_label': coarse1_label
        }

In [17]:
model = VGGCustomModel(vgg16_bn, classes_per_coarse)

In [18]:
class CustomCategorize(DisplayedTransform):
    "Reversible transform of category string to `vocab` id"
    loss_func,order=CrossEntropyLossFlat(),1
    def __init__(self, vocab=None, vocab_coarse1=None, vocab_coarse2=None, sort=True, add_na=False, num_y=1):
        store_attr()
        self.vocab = None if vocab is None else CategoryMap(vocab, sort=sort, add_na=add_na)
        self.vocab_coarse1 = None if vocab_coarse1 is None else CategoryMap(vocab_coarse1, sort=sort, add_na=add_na)

    def setups(self, dsets):
        fine_dsets = [d[1] for d in dsets]
        coarse1_dsets = [d[0] for d in dsets]
        if self.vocab is None and dsets is not None: self.vocab = CategoryMap(fine_dsets, sort=self.sort, add_na=self.add_na)
        if self.vocab_coarse1 is None and dsets is not None: self.vocab_coarse1 = CategoryMap(coarse1_dsets, sort=self.sort, add_na=self.add_na)
        self.c = len(self.vocab)

    def encodes(self, o): return {'fine_label': TensorCategory(self.vocab.o2i[o[1]]),
                                  'coarse1_label': TensorCategory(self.vocab_coarse1.o2i[o[0]])
                                 }
    def decodes(self, o): return Category      (self.vocab    [o])

In [19]:
def CustomCategoryBlock(vocab=None, sort=True, add_na=False, num_y=1):
    "`TransformBlock` for single-label categorical targets"
    return TransformBlock(type_tfms=CustomCategorize(vocab=vocab, sort=sort, add_na=add_na))

In [20]:
def custom_splitter(model):
    return [params(model.shared_layers),
            params(model.coarse_component),
            params(model.fine_components),
            params(model.coarse_head),
            params(model.fine_heads)]

In [21]:
fishes = DataBlock(blocks = (ImageBlock, CustomCategoryBlock),
                 splitter=ColSplitter(),
                 get_x = ColReader(5, pref=path),
                 get_y=custom_get_y,
                 item_tfms=RandomResizedCrop(336, min_scale=0.5),
                 batch_tfms=aug_transforms())
dls = fishes.dataloaders(df)

In [22]:
dls.train_ds, dls.valid_ds

((#69110) [(PILImage mode=RGB size=768x1024, {'fine_label': TensorCategory(62), 'coarse1_label': TensorCategory(1)}),(PILImage mode=RGB size=1024x768, {'fine_label': TensorCategory(58), 'coarse1_label': TensorCategory(2)}),(PILImage mode=RGB size=768x1024, {'fine_label': TensorCategory(79), 'coarse1_label': TensorCategory(0)}),(PILImage mode=RGB size=1024x768, {'fine_label': TensorCategory(52), 'coarse1_label': TensorCategory(1)}),(PILImage mode=RGB size=768x1024, {'fine_label': TensorCategory(6), 'coarse1_label': TensorCategory(2)}),(PILImage mode=RGB size=1024x768, {'fine_label': TensorCategory(69), 'coarse1_label': TensorCategory(0)}),(PILImage mode=RGB size=1024x715, {'fine_label': TensorCategory(130), 'coarse1_label': TensorCategory(1)}),(PILImage mode=RGB size=1024x768, {'fine_label': TensorCategory(108), 'coarse1_label': TensorCategory(1)}),(PILImage mode=RGB size=1024x768, {'fine_label': TensorCategory(118), 'coarse1_label': TensorCategory(0)}),(PILImage mode=RGB size=1024x768,

In [23]:
weights = tensor([float(weights_df[weights_df['Specie']==c]['Weight']) for c in dls.vocab]).cuda()
weights_family = tensor([float(weights_family_df[weights_family_df['Family']==c]['Weight']) for c in dls.vocab_coarse1]).cuda()

In [24]:
def loss_func(out, targ):
    return nn.CrossEntropyLoss(weight=weights)(out['fine_label'], targ['fine_label']) + \
            nn.CrossEntropyLoss(weight=weights_family)(out['coarse1_label'], targ['coarse1_label'])

In [25]:
def custom_accuracy(inp, targ, axis=-1):
    pred1,targ1 = flatten_check(inp['fine_label'].argmax(dim=axis), targ['fine_label'])
    acc1 = (pred1 == targ1).float().mean()
    return acc1

In [30]:
learn = Learner(dls, model, loss_func=loss_func, metrics=custom_accuracy,
                   splitter=custom_splitter).to_fp16()
learn.freeze()

In [31]:
learn.summary()

VGGCustomModel (Input shape: ['64 x 3 x 336 x 336'])
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               64 x 64 x 336 x 336  1,792      False     
________________________________________________________________
BatchNorm2d          64 x 64 x 336 x 336  128        True      
________________________________________________________________
ReLU                 64 x 64 x 336 x 336  0          False     
________________________________________________________________
Conv2d               64 x 64 x 336 x 336  36,928     False     
________________________________________________________________
BatchNorm2d          64 x 64 x 336 x 336  128        True      
________________________________________________________________
ReLU                 64 x 64 x 336 x 336  0          False     
________________________________________________________________
MaxPool2d            64 x 64 x 168 x 168  0          False     
_____________________________________________

In [32]:
learn.fit(1, 3e-3)

epoch,train_loss,valid_loss,custom_accuracy,time
0,3.888104,3.586677,0.239322,10:04


In [33]:
learn.save('species144-hdcnn-vgg16-vgg16-fepochs1-uepochs0')

Path('models/species144-hdcnn-vgg16-vgg16-fepochs1-uepochs0.pth')

In [26]:
def continue_training(pth_filename):
    learn = Learner(dls, model, loss_func=loss_func, metrics=custom_accuracy,
                   splitter=custom_splitter).to_fp16()
    learn.load(pth_filename)
    learn.unfreeze()
    return learn

In [35]:
learn.unfreeze()
learn.fit(10, slice(1e-6,1e-4))

epoch,train_loss,valid_loss,custom_accuracy,time
0,3.208007,2.773470,0.363121,12:08
1,2.912249,2.511503,0.396053,12:05
2,2.706429,2.359719,0.420708,12:06
3,2.548780,2.230437,0.438072,12:06
4,2.370211,2.131509,0.455087,12:05
5,2.335464,2.030832,0.478933,12:06
6,2.202054,1.951346,0.489351,12:05
7,2.139999,1.896795,0.504051,12:05
8,2.056375,1.840777,0.512559,12:06
9,1.924729,1.793004,0.521125,12:03


In [36]:
learn.save('species144-hdcnn-vgg16-fepochs1-uepochs10')

Path('models/species144-hdcnn-vgg16-vgg16-fepochs1-uepochs10.pth')

In [27]:
learn = continue_training('species144-hdcnn-vgg16-fepochs1-uepochs10')

In [28]:
learn.fit(10, slice(1e-6,1e-4))
learn.save('species144-hdcnn-vgg16-fepochs1-uepochs20')

epoch,train_loss,valid_loss,custom_accuracy,time
0,1.925967,1.760417,0.530154,12:05
1,1.831533,1.715354,0.540456,12:02
2,1.789760,1.675744,0.547401,12:02
3,1.751735,1.643864,0.551048,12:02
4,1.703548,1.604352,0.555504,12:03
5,1.706680,1.557171,0.563491,12:02
6,1.544008,1.536182,0.569163,12:03
7,1.469456,1.511372,0.574661,12:03
8,1.436826,1.499674,0.581028,12:03
9,1.469738,1.466440,0.585369,12:03


Path('models/species144-hdcnn-vgg16-fepochs1-uepochs20.pth')

In [27]:
learn = continue_training('species144-hdcnn-vgg16-fepochs1-uepochs20')

In [28]:
learn.fit(10, slice(1e-6,1e-4))
learn.save('species144-hdcnn-vgg16-fepochs1-uepochs30')

epoch,train_loss,valid_loss,custom_accuracy,time
0,1.447829,1.442144,0.597812,11:59
1,1.439801,1.432475,0.599722,12:00
2,1.413651,1.421753,0.596828,11:56
3,1.331459,1.392815,0.604294,11:57
4,1.302406,1.386604,0.601864,11:58
5,1.280561,1.361318,0.614944,11:59
6,1.251222,1.347972,0.614654,12:00
7,1.230428,1.336396,0.619227,11:59
8,1.185642,1.333822,0.621252,11:58
9,1.206094,1.332209,0.625072,11:59


Path('models/species144-hdcnn-vgg16-fepochs1-uepochs30.pth')

In [27]:
learn = continue_training('species144-hdcnn-vgg16-fepochs1-uepochs30')

In [28]:
learn.fit(10, slice(1e-6,1e-4))
learn.save('species144-hdcnn-vgg16-fepochs1-uepochs40')

epoch,train_loss,valid_loss,custom_accuracy,time
0,1.182881,1.333762,0.629760,12:29
1,1.196474,1.306304,0.631207,12:26
2,1.092125,1.298122,0.638789,12:28
3,1.115962,1.287490,0.637632,12:29
4,1.042941,1.277644,0.647587,12:29
5,1.030285,1.264100,0.641799,12:30
6,0.994348,1.253383,0.641278,12:28
7,0.980368,1.252073,0.653606,12:27
8,1.019139,1.249696,0.648397,12:29
9,0.926638,1.242325,0.653259,12:28


Path('models/species144-hdcnn-vgg16-fepochs1-uepochs40.pth')

In [27]:
learn = continue_training('species144-hdcnn-vgg16-fepochs1-uepochs40')

In [28]:
learn.fit(10, slice(1e-6,1e-4))
learn.save('species144-hdcnn-vgg16-fepochs1-uepochs50')

epoch,train_loss,valid_loss,custom_accuracy,time
0,0.993170,1.258228,0.655631,12:15
1,0.941892,1.254346,0.658062,12:12
2,0.953662,1.230761,0.655631,12:10
3,0.887695,1.222708,0.667381,12:09
4,0.876926,1.219270,0.661593,12:07
5,0.909806,1.216929,0.665065,12:07
6,0.836628,1.226028,0.661709,12:09
7,0.838147,1.219545,0.665586,12:09
8,0.818376,1.217786,0.665934,12:10
9,0.833712,1.208763,0.670564,12:09


Path('models/species144-hdcnn-vgg16-fepochs1-uepochs50.pth')

In [ ]:
learn.export('species144-hdcnn-vgg16-fepochs1-uepochs50')